# Preload precipitation files

Since loading from the original COSMO GRIB files takes quite a long time, I will try to preload them in a separate file. This should hopefully speed up my analysis.

In [38]:
# Imports
from helpers import *
from config import *
import os
import numpy as np
import datetime as dt
from cosmo_utils.helpers import ddhhmmss, make_timelist, yyyymmddhhmmss_strtotime, yyyymmddhhmmss
import timeit

In [35]:
exp_ids = ['DA_REF', 'DA_PSPv2', 'DA_REF_TL500', 'DA_PSPv2_TL500', 'noDA_PSPv2']
date_start = '20160526000000'
date_stop = '20160608120000'
hint = 12
fclt = 24   # Forecast lead time

In [32]:
date_list = make_timelist(
    yyyymmddhhmmss_strtotime(date_start),
    yyyymmddhhmmss_strtotime(date_stop),
    dt.timedelta(hours=hint),
)

In [34]:
date_list[:5]

[datetime.datetime(2016, 5, 26, 0, 0),
 datetime.datetime(2016, 5, 26, 12, 0),
 datetime.datetime(2016, 5, 27, 0, 0),
 datetime.datetime(2016, 5, 27, 12, 0),
 datetime.datetime(2016, 5, 28, 0, 0)]

## Preload deterministic forecasts

In [40]:
def preload_det_prec_for_one_day(exp_id, date_str, fclt=24):
    data_dir = datadir + exp_id + '/'
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        p = load_det(data_dir, date_str, hstr, return_array=True);
        if h == 1:  # Allocate output array
            prec = np.empty((fclt, p.shape[0], p.shape[1]))
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'det_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [41]:
preload_det_prec_for_one_day('DA_REF', date_start)

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526

## Preload ensemble forecasts

In [46]:
def preload_ens_prec_for_one_day(exp_id, date_str, fclt=24):
    data_dir = datadir + exp_id + '_ens/'
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        p = np.array(load_ens(data_dir, date_str, hstr, return_array=True))
        if h == 1:  # Allocate output array
            prec = np.empty((fclt, p.shape[0], p.shape[1], p.shape[2]))
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'ens_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [47]:
preload_ens_prec_for_one_day('DA_REF', date_start)

read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:03.646483
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.738707
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:06.395695
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.334186
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.423728
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.442428
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.723308
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.542957
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
re

## Loop over experiments and dates

In [ ]:
for exp_id in exp_ids:
    print exp_id
    for date in date_list:
        print date
        date_str = yyyymmddhhmmss(date)
        preload_det_prec_for_one_day(exp_id, date_str)
        preload_ens_prec_for_one_day(exp_id, date_str)

DA_REF
2016-05-26 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/da